In [ ]:
#import neg_score and pos_score
import pickle
import numpy as np
import matplotlib.pyplot as plt
# run_name =  'rm_hh_rlhf_202411100543'
from gpt import GPT, GPTRewardModel
from configs import get_configs
from dataset import StepDPODataset
import torch
from loss import KPairwiseLoss, CrossEntropyLoss, ValueLoss, PolicyLoss
import statistics
import datasets
cfg = get_configs('gpt2-medium')
import tiktoken

In [ ]:
#for Math
sft_model = GPT.from_checkpoint(cfg, 'runs\sft_metaMath_202412210335\sft_metaMath_202412210335_step161558.pt')
ppo_model = GPT.from_checkpoint(cfg, 'runs\ppo_metaMath2212_202412221836\ppo_metaMath2212_202412221836_actor_step500.pt')

In [ ]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_math_gpt2(model, prompt, device, samples=100):
    model.eval()
    model.to(device)
    max_new_tokens = 512
    temperature = 0.2
    top_k = 50
    x, decode = prepare_gpt2_input(prompt, device)
    list_ans = dict()
    cnt=0
    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        s = decode(y[0].tolist())
        try:
            ori_ans = s[s.index('The answer is') + len('The answer is'):].strip()
            ans = ori_ans[:ori_ans.index('<|endoftext|>')]
            ans = ans.strip()
            if ans not in list_ans:
                list_ans[ans] = 1
            else:
                list_ans[ans]+=1
        except Exception as e:
            cnt+=1
            print('Error: ', e)
            print(s)
    print(f'{cnt} samples with unknown answer')
    return list_ans

In [ ]:
dataset = datasets.load_dataset('gsm8k', "main")
train_dataset, test_dataset = dataset['train'], dataset['test']
question = 'problem'
answer = 'answer'

In [ ]:
abs_error1 = 0
abs_error2 = 0
plot_error1 = []
plot_error2 = []
for i in range(10):
    prompt = train_dataset[i][question]
    print(prompt)
    ans_sft = generate_math_gpt2(sft_model, prompt, 'cuda', samples=10)
    sorted_ans = dict(sorted(ans_sft.items(), key=lambda item: -item[1]))
    #get first answer
    ans = list(sorted_ans.keys())[0]
    print(ans)
    print(generate_math_gpt2(ppo_model, prompt, 'cuda', samples=10))
    sorted_ans = dict(sorted(ans_sft.items(), key=lambda item: -item[1]))
    #get first answer
    ans2 = list(sorted_ans.keys())[0]
    print(ans2)
    true_ans = train_dataset[i][answer]
    true_ans = true_ans[true_ans.rindex(r'####') + 4:].strip()
    abs_error1 += abs(int(ans) - int(true_ans))
    abs_error2 += abs(int(ans2) - int(true_ans))
    print('True answer: ', true_ans)
    print('Answer 1: ', ans)
    print('Answer 2: ', ans2)
    print('------------------------------------')
    plot_error1.append(abs(int(ans) - int(true_ans)))
    plot_error2.append(abs(int(ans2) - int(true_ans)))
plt.plot(plot_error1, label='Error 1')
plt.plot(plot_error2, label='Error 2')
print('Average error 1: ', abs_error1/10)
print('Average error 2: ', abs_error2/10)